In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv('Datasets\Microsoft_Azure_Predictive_Maintenance\PdM_telemetry.csv')
df.head()

,datetime,machineID,volt,rotate,pressure,vibration
0,2015-01-01 06:00:00,1,176.217853,418.504078,113.077935,45.087686
1,2015-01-01 07:00:00,1,162.879223,402.747490,95.460525,43.413973
2,2015-01-01 08:00:00,1,170.989902,527.349825,75.237905,34.178847
3,2015-01-01 09:00:00,1,162.462833,346.149335,109.248561,41.122144
4,2015-01-01 10:00:00,1,157.610021,435.376873,111.886648,25.990511


In [5]:
failure = pd.read_csv('Datasets\Microsoft_Azure_Predictive_Maintenance\PdM_failures.csv')
failure

,datetime,machineID,failure
0,2015-01-05 06:00:00,1,comp4
1,2015-03-06 06:00:00,1,comp1
2,2015-04-20 06:00:00,1,comp2
3,2015-06-19 06:00:00,1,comp4
4,2015-09-02 06:00:00,1,comp4
...,...,...,...
756,2015-11-29 06:00:00,99,comp3
757,2015-12-14 06:00:00,99,comp4
758,2015-02-12 06:00:00,100,comp1
759,2015-09-10 06:00:00,100,comp1


In [9]:
ID = failure['machineID'].unique()
count = {}
for i in ID:
    count[i] = failure[failure['machineID']==i].shape[0]

In [11]:
ID

array([  1,   2,   3,   4,   5,   7,   8,   9,  10,  11,  12,  13,  14,
        15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
        28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,
        41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,
        54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,
        67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  78,  79,  80,
        81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,
        94,  95,  96,  97,  98,  99, 100])

In [13]:
count

{np.int64(1): 7,
 np.int64(2): 4,
 np.int64(3): 5,
 np.int64(4): 6,
 np.int64(5): 7,
 np.int64(7): 10,
 np.int64(8): 5,
 np.int64(9): 9,
 np.int64(10): 5,
 np.int64(11): 6,
 np.int64(12): 6,
 np.int64(13): 13,
 np.int64(14): 4,
 np.int64(15): 7,
 np.int64(16): 13,
 np.int64(17): 15,
 np.int64(18): 8,
 np.int64(19): 7,
 np.int64(20): 13,
 np.int64(21): 11,
 np.int64(22): 15,
 np.int64(23): 12,
 np.int64(24): 10,
 np.int64(25): 8,
 np.int64(26): 6,
 np.int64(27): 3,
 np.int64(28): 5,
 np.int64(29): 2,
 np.int64(30): 5,
 np.int64(31): 7,
 np.int64(32): 10,
 np.int64(33): 9,
 np.int64(34): 3,
 np.int64(35): 11,
 np.int64(36): 4,
 np.int64(37): 14,
 np.int64(38): 7,
 np.int64(39): 4,
 np.int64(40): 9,
 np.int64(41): 3,
 np.int64(42): 9,
 np.int64(43): 10,
 np.int64(44): 4,
 np.int64(45): 7,
 np.int64(46): 2,
 np.int64(47): 10,
 np.int64(48): 7,
 np.int64(49): 12,
 np.int64(50): 4,
 np.int64(51): 10,
 np.int64(52): 7,
 np.int64(53): 6,
 np.int64(54): 5,
 np.int64(55): 8,
 np.int64(56): 11,
 

In [14]:
df1 = df[df['machineID']==1]
df1.head()

,datetime,machineID,volt,rotate,pressure,vibration
0,2015-01-01 06:00:00,1,176.217853,418.504078,113.077935,45.087686
1,2015-01-01 07:00:00,1,162.879223,402.747490,95.460525,43.413973
2,2015-01-01 08:00:00,1,170.989902,527.349825,75.237905,34.178847
3,2015-01-01 09:00:00,1,162.462833,346.149335,109.248561,41.122144
4,2015-01-01 10:00:00,1,157.610021,435.376873,111.886648,25.990511


In [15]:
failure[failure['machineID']==1].head()

,datetime,machineID,failure
0,2015-01-05 06:00:00,1,comp4
1,2015-03-06 06:00:00,1,comp1
2,2015-04-20 06:00:00,1,comp2
3,2015-06-19 06:00:00,1,comp4
4,2015-09-02 06:00:00,1,comp4
